In [11]:
import cv2
import tkinter as tk
from tkinter import filedialog, Label, Frame, messagebox
from PIL import Image, ImageTk

class SketchApp:
    def __init__(self, root):
        # Initialize the main window and set layout
        self.root = root
        self.root.title("Pencil Sketch Converter")
        self.root.geometry("800x600")  # Initial window size
        
        # Variables to store the image path and sketch image
        self.loaded_image_path = None
        self.sketch_image = None

        # Create the layout
        self.create_layout()

    def create_layout(self):
        # Create a main frame
        self.frame = Frame(self.root, bg="lightblue")
        self.frame.grid(row=0, column=0, padx=10, pady=10, sticky="nsew")

        # Configure grid layout to be responsive
        self.root.grid_columnconfigure(0, weight=1)
        self.root.grid_rowconfigure(0, weight=1)
        self.frame.grid_columnconfigure(0, weight=1)
        self.frame.grid_rowconfigure(0, weight=9)
        self.frame.grid_rowconfigure(1, weight=1)

        # Create a label widget to display the image
        self.image_label = Label(self.frame, bg="white")
        self.image_label.grid(row=0, column=0, sticky="nsew", padx=10, pady=10)

        # Create a frame for buttons
        button_frame = Frame(self.frame, bg="lightgray")
        button_frame.grid(row=1, column=0, sticky="ew", pady=5)

        # Button to load the image
        load_button = tk.Button(button_frame, text="Open Image", command=self.open_file, font=("Helvetica", 14), bg="blue", fg="white", padx=10, pady=5)
        load_button.pack(side="left", padx=10)

        # Arrow button to convert the image to a sketch
        arrow_button = tk.Button(button_frame, text="➡ Convert to Sketch", command=self.show_sketch, font=("Helvetica", 14), bg="green", fg="white", padx=10, pady=5)
        arrow_button.pack(side="left", padx=10)

        # Button to save the sketch as PNG
        save_button = tk.Button(button_frame, text="💾 Save Sketch", command=self.save_sketch, font=("Helvetica", 14), bg="orange", fg="white", padx=10, pady=5)
        save_button.pack(side="left", padx=10)

        # Bind window resize event
        self.root.bind("<Configure>", self.resize_on_window)

    def open_file(self):
        
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png")])
        if file_path:
            self.loaded_image_path = file_path
            self.sketch_image = None  # Reset sketch image
            self.display_image(file_path)

    def display_image(self, img_path, is_sketch=False):
        # Load the image and resize it
        if is_sketch and self.sketch_image is not None:
            img = self.sketch_image
        else:
            img = cv2.imread(img_path)
        
        resized_img = self.resize_image(img, self.image_label.winfo_width(), self.image_label.winfo_height())
        
        # Convert image to a format tkinter can display
        img = Image.fromarray(cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB) if not is_sketch else resized_img)
        img = ImageTk.PhotoImage(image=img)

        # Update the image label with the new image
        self.image_label.config(image=img)
        self.image_label.image = img  # Keep a reference to avoid garbage collection

    def show_sketch(self):
        if self.loaded_image_path:
            self.sketch_image = self.convert_to_sketch(self.loaded_image_path)
            self.display_image(self.loaded_image_path, is_sketch=True)

    def convert_to_sketch(self, img_path):
        image = cv2.imread(img_path)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        inverted_image = 255 - gray_image
        blurred = cv2.GaussianBlur(inverted_image, (21, 21), 0)
        inverted_blur = 255 - blurred
        sketch = cv2.divide(gray_image, inverted_blur, scale=256.0)
        return sketch

    def resize_image(self, image, width, height):
        # Resize the image to fit the window while keeping the aspect ratio
        h, w = image.shape[:2]
        aspect_ratio = w / h
        if width / height > aspect_ratio:
            new_height = height
            new_width = int(aspect_ratio * new_height)
        else:
            new_width = width
            new_height = int(new_width / aspect_ratio)
        return cv2.resize(image, (new_width, new_height))

    def save_sketch(self):
        # Save the sketch as a PNG file
        if self.sketch_image is not None:
            save_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png")])
            if save_path:
                cv2.imwrite(save_path, self.sketch_image)
                messagebox.showinfo("Success", "Sketch saved successfully!")
        else:
            messagebox.showwarning("No Sketch", "You need to convert an image to a sketch first!")

    def resize_on_window(self, event):
        # Resize the image when the window is resized
        if self.loaded_image_path:
            self.display_image(self.loaded_image_path, is_sketch=self.sketch_image is not None)

# Create the main window
root = tk.Tk()

# Instantiate the application class
app = SketchApp(root)

# Start the main event loop
root.mainloop()
